In [76]:
import os
import numpy as np
import json
import random
from scipy.spatial.transform import Rotation

In [77]:
SAVE_PATH = os.path.join(os.getcwd(), '../json')
print(SAVE_PATH)
FILE_NAME = 'camera_path.json'

/home/boomalope/nerf_estimator/src/nerf_estimator/camera_path/scripts/../json


In [78]:
path = {}
path["camera_type"] = "perspective"
path["render_height"] = 1080//2
path["render_width"] = 1920//2
path["fps"] = 24
path["seconds"] = 1
path["smoothness_value"] = 0.5
path["is_cycle"] = False
path["crop"] = None

In [79]:
z = 0
pitch = 0.7
fov = 50
aspect = 1

canmera_path = []

#Scene bounded by a box with length of 4 centrered at (0, 0, 0)
#Scale relative to maxima of bounds
x = 0
y = -0.36 

transform_matrix = np.eye(4)
transform_matrix[0, 3] = x
transform_matrix[1, 3] = y
transform_matrix[2, 3] = z
transform_matrix[:3, :3] = Rotation.from_euler('xyz', [pitch, 0, 0]).as_matrix()

#Random walk
for _ in range(path["fps"] * path["seconds"]):
    canmera_path.append({
        "camera_to_world": transform_matrix.flatten().tolist(),
        "fov": fov,
        "aspect": aspect
    })

    #Rotate by random yaw
    #yaw = random.normalvariate(0.05, 0.1)
    yaw = 0.05
    rot = Rotation.from_rotvec(np.array([0, 0, yaw])).as_matrix()
    rot_matrix = np.eye(4)
    rot_matrix[:3, :3] = rot 
    transform_matrix = rot_matrix @ transform_matrix

    #Move by random x, y
    #dx = random.normalvariate(0, 0.001)
    dx = 0
    #dy = random.normalvariate(0, 0.001)
    dy = 0
    transform_matrix[0, 3] += dx
    transform_matrix[1, 3] += dy

path["camera_path"] = canmera_path

In [80]:
#Save json
with open(os.path.join(SAVE_PATH, FILE_NAME), "w") as f:
    json.dump(path, f, indent=4)